In [ ]:
using CSV, CurveFit
using Dates, DataFrames
using Glob
using Dates, DataFrames, Distributions, DSP
using NativeFileDialog
using LaTeXStrings
using Polynomials, Printf
using Statistics #, StatsPlotss
using StatsBase

display(HTML("<style>.jp-Cell { width: 150% !important; }</style>"))
##include("./Split_Spectra_Tools.jl")

# Function for splitting filename and extracting details
function extract_details(infil)
###############################
    
    split_details = split(split(infil, "\\")[end], ".")[1]
    site_name = split_details[1:4]
    date_string = split_details[6:15]
    time_string = replace(split_details[17:21], 'h' => ':')
    date = DateTime(date_string * " " * time_string, "yyyy-mm-dd HH:MM")
    date_string*" "*time_string

    return(site_name, date, date_string*" "*time_string)
    
end    # extract_details()


function get_displacements(arry)
#####################################
    
    displacements = []

    if length(arry[1]) == 3
    
        for i ∈ arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
        end
        
    else
        
        for i ∈ arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
        end
        
    end

    displacements[findall(>=(2048), displacements)] = 2048 .- displacements[findall(>=(2048), displacements)];
    
    return(displacements./100)
    
    end     # get_displacements()


function process_spectrum_file(df)
################################
    
    is_gps = false
    sync_word_location = findall(x -> x == "7FFF", df.Column2)

    for j ∈ sync_word_location

        try
            
            i = df.Column2[j+1]
            word_number = parse(Int, SubString.(i, 1, 1), base=16)*16^0
            word = parse(Int, SubString.(i, 2, 2), base=16)*16^2 + parse(Int, SubString.(i, 3, 3), base=16)*16^1 + parse(Int, SubString.(i, 4, 4), base=16)*16^0
##            println(j,' ',word_number,' ',word)
    
            # Test whether buoy is MkIII or DWR-G - see p.51 Table 5.7.5a. Organization and significance of the system file data 
            # If Av0 = 0; Ax0 = 0; Ay0 = 0; O = 0; and Inclination = 0 then it's a DWR-G
            
            if (word_number == 7 && word == 0)
                is_gps = true
            end

        catch

            #do nothing

        end
            
    end
    
    return(is_gps)
    
    end    # process_spectrum_file()


function fix_gps_errors(heave, date)
# function to apply polynomial fit to WSE's affected by GPS errors
# uses selectable offset value to fine-tune result
#####################################
    gps_errors = findall(isodd,parse.(Int,SubString.(string.(df.Column4), 2, 2), base = 16))

    heave_length = length(heave)
    
    if !isempty(gps_errors)
        
        println("\n",length(gps_errors)," GPS errors at ", Dates.format.(date, "yyyy-mm-dd HH:MM"))
        
        for ii ∈ reverse(gps_errors)

            error_center = ii

            # User-selected offset either side of GPS error
            lower_offset = upper_offset = 120

            if error_center <= lower_offset
                lower_offset = error_center - 1
            end

            if error_center+upper_offset > heave_length
                upper_offset = heave_length - error_center
            end

            # Ensure there are at least 3 points for fitting
            lower_offset = max(lower_offset, 2)
            upper_offset = max(upper_offset, 2)
    
            # Fit curve to subset of heave before GPS error
            left_side_points = error_center-lower_offset:error_center
            fit1 = curve_fit(Polynomial, left_side_points, heave[left_side_points], 2)
            yfit1 = fit1.(left_side_points)
            yfit1[length(yfit1)] = 0.0

            # Fit curve to subset of heave after GPS error
            right_side_points = error_center:error_center+upper_offset
            fit2 = curve_fit(Polynomial, right_side_points, heave[right_side_points], 2)
            yfit2 = fit2.(right_side_points)
            yfit2[1] = 0.0

            # apply polynomial results to wse's on both sides of GPS error
            heave[left_side_points] .= heave[left_side_points] - yfit1
            heave[right_side_points] .= heave[right_side_points] - yfit2
            heave[ii] = 0.0    # set wse at GPS error location to 0

        end
    
    end

    return(heave)
    
    end     # fix_gps_errors()


# function that seeks to locate and repair those GPS errors where no GPS flag has been reported
function no_flag_fix(heave, date)
#################################

    rec_length = length(heave)

##    println(date)
    
    heave_time = date .+ (0:length(heave)-1) .* Microsecond.(1/1.28 * 1000000)
    
    # Locate maximum run of negative values + 1 to identify center of GPS error
    negatives = findall(x -> x < 0, heave)
    error_center = findfirst(x -> x == heave_time[(negatives)[findmax(diff(negatives))[2]] + 1], heave_time)
    ii = error_center
    
    # User-selected offset either side of GPS error
    lower_offset = upper_offset = 50
    
    if error_center <= lower_offset
        lower_offset = error_center - 1
    end
    
    if error_center + upper_offset > rec_length
        upper_offset = rec_length - error_center
    end
    
    # Ensure there are at least 3 points for fitting
    lower_offset = max(lower_offset, 2)
    upper_offset = max(upper_offset, 2)
    
    # Fit curve to subset of heave before GPS error
    left_side_points = max(1, error_center - lower_offset):error_center
    if length(left_side_points) >= 3
        fit1 = curve_fit(Polynomial, left_side_points, heave[left_side_points], 2)
        yfit1 = fit1.(left_side_points)
        yfit1[length(yfit1)] = 0.0
        heave[left_side_points] .= heave[left_side_points] - yfit1
    end
    
    # Fit curve to subset of heave after GPS error
    right_side_points = error_center:min(error_center + upper_offset, rec_length)
    if length(right_side_points) >= 3
        fit2 = curve_fit(Polynomial, right_side_points, heave[right_side_points], 2)
        yfit2 = fit2.(right_side_points)
        yfit2[1] = 0.0
        heave[right_side_points] .= heave[right_side_points] - yfit2
    end
    
    # Set wse at GPS error location to 0
    heave[ii] = 0.0
    
    return(heave)

end    # no_flag_fix()


# extract the displacements from the HEX data
function get_HNW(infil, date)
#####################################
        
    global df = DataFrame(CSV.File(infil,header=0, delim=",", types=String));

    # Remove rows that do not match Datawell's data format
##    df = df[Bool[length(x) == 24 for x in df.Column1], :]

    is_gps = process_spectrum_file(df)
##    println(is_gps)
    
    # Calculate sequence numbers
    arry = SubString.(df.Column1, 3, 4)

    sequence = []

    for i ∈ arry
        append!(sequence,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
    end

    # Calculate heave WSEs
    arry = SubString.(df.Column3, 1, 3);
    heave = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column3, 4, ) .* SubString.(df.Column4, 1, 2)
    north = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column4, 3, 4) .* SubString.(df.Column5, 1, 1)
    west = get_displacements(arry);

    if is_gps

        heave = fix_gps_errors(heave, date)
        heave = no_flag_fix(heave, date)

    end

    return(heave, north, west)

    end    # get_HNW()


# smooth the spectra into bands centered on 0.05Hz spacing (i.e. 0:0.005:0.64)
function smooth_spectra(Pden_in, sample_frequency)
##################################################

    nyquist = sample_frequency/2

    freq_in = range(0, stop=nyquist, length=length(Pden_in))

    freq_out = [0.0]
    Pden_smoothed = [mean(Pden_in[1:8])]

    i = 9
    while i <= length(Pden_in)

        push!(freq_out,freq_in[i+8])

        if i < length(Pden_in)-16

            push!(Pden_smoothed, mean(Pden_in[i:i+16]))

        end

        i+=16

    end

    push!(Pden_smoothed, mean(Pden_in[end-8:end]))
            
    return (freq_out, Pden_smoothed)
        
end    # smooth_spectra()


function get_Fourier_coefficients(heave, north, west)
#####################################################    

    # Get the cross periodograms
    cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);
    cps_north_north = mt_cross_power_spectra([north north]', fs=sample_frequency);
    cps_west_west = mt_cross_power_spectra([west west]', fs=sample_frequency);

    cps_north_heave = mt_cross_power_spectra([north heave]', fs=sample_frequency);
    cps_west_heave = mt_cross_power_spectra([west heave]', fs=sample_frequency);
    cps_north_west = mt_cross_power_spectra([north west]', fs=sample_frequency);

##    fhh = cps_heave_heave.freq
    fhh, Chh = smooth_spectra(real.(cps_heave_heave.power[1,1,:]), sample_frequency)

    #fnn = cps_north_north.freq
    fhh, Cnn = smooth_spectra(real.(cps_north_north.power[1,1,:]), sample_frequency)

    #fww = cps_west_west.freq
    fhh, Cww = smooth_spectra(real.(cps_west_west.power[1,1,:]), sample_frequency)

    #fnw = cps_north_west.freq
    fhh, Cnw = smooth_spectra(real.(cps_north_west.power[1,2,:]), sample_frequency)

    #fnh = cps_north_heave.freq
    fhh, Qnh = smooth_spectra(imag.(cps_north_heave.power[1,2,:]), sample_frequency)

    #fwh = cps_west_heave.freq
    fhh, Qwh = smooth_spectra(imag.(cps_west_heave.power[1,2,:]), sample_frequency)

    a1 = Qnh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5
    b1 = -Qwh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5

    a2 = (Cnn .- Cww) ./ (Cnn .+ Cww)
    b2 = -2 .* Cnw ./ (Cnn .+ Cww)
    
    return(fhh, Chh, a1, b1, a2, b2)
    
end    # get_Fourier_coefficients()


function calc_f2_Pden2(heave)
#############################
    
    # Show spectra using Welch's method to better define bimodal events
    ps_w = welch_pgram(heave, 256, 128; onesided=true, nfft=256, fs=sample_frequency, window=hanning);
    f2 = freq(ps_w);
    Pden2 = power(ps_w)    

    return(f2, Pden2)

end    # calc_f2_Pden2()
    

function calc_spread_direction(a1,b1)
#####################################
    
    θ₀ = atan.(b1,a1)
    m1 = (a1.^2 .+ b1.^2).^0.5
##    m2 = a2.*cos.(2*θ₀) .+ b2.*sin.(2*θ₀)
##    n2 = -a2.*sin.(2*θ₀) .+ b2.*cos.(2*θ₀)

    # Calculate the spread
    σc = (2 .* (1 .- m1)).^0.5;

    direction = mod.(rad2deg.(atan.(b1,a1)),360)

    return(σc, direction)

end    # calc_spread_direction()


# Function to replace NaN with 0 in the specified column of the df
function replace_nan_with_zero!(df::DataFrame, col::Symbol)

    df[!, col] = [replace(x -> isnan(x) ? 0 : x, df[!, col][i]) for i in 1:nrow(df)]

end    # replace_nan_with_zero!()


#######################################################################################################
#######################################################################################################
#######################################################################################################

using Base.Filesystem: stat

#include("./Split_Spectra_Tools.jl")

const sample_frequency = 1.28

# Define the path to the directory you want to search in
directory_path = pick_folder()

# Use glob to find all .HXV files in the selected directory and its subdirectories
hxv_files = glob("**/*.hxv", directory_path)

# build df containing displacements and Fourier coefficients for selected day
displacement_df = DataFrame(Date = [], Heave = [], North = [], West = [], fhh = [], Chh = [], 
    a1 = [], b1 = [], a2 = [], b2 = [], f2 = [], Pden2 = [], Spread = [], Direction = [])

println("Reading all .HXV files for selected month (this takes a while!):")
flush(stdout)

for infil ∈ hxv_files

    # check for empty files
    file_stat = stat(infil)
    if file_stat.size > 0
        
        site_name, date, date_string = extract_details(infil)
    
        print(".")
            
        heave, north, west = get_HNW(infil, date)
                         
        # get frequencies, spectra, and Fourier coefficients
        fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)
    
        f2, Pden2 = calc_f2_Pden2(heave)
        σc, direction = calc_spread_direction(a1,b1)
    
        push!(displacement_df, (date, heave, north, west, fhh, Chh, a1, b1, a2, b2, f2, Pden2, σc, direction))
        
    else
        
        print("x")

    end

end

# Replace NaN with 0
replace_nan_with_zero!(displacement_df, :Pden2)

println("\nDone!")

### Perform the Sea-Swell split

In [ ]:
using DataFrames
using EasyFit
using Peaks, Plots


# Function to find local maxima and minima
function find_local_extrema(pden2)
##################################
    
    local_maxima = argmaxima(pden2, 3; strict=true)
    local_minima = argminima(pden2, 3; strict=true)
    
    return(local_maxima, local_minima)
    
end    # find_local_extrema()


# Calculate a PM spectrum for a given peak frequency
function pierson_moskowitz_spectrum(f, fp)
##########################################
    
    α = 0.0081
    g = 9.81
    
    return(α * g^2 * (2π)^-4 * f .^ -4 .* exp.(-5/4 * (fp ./ f) .^ 4))
    
end    # pierson_moskowitz_spectrum()

# Check for significant low-frequency energy
function check_low_frequency_energy(pden2, f2, threshold=0.02)
##############################################################
    
    low_freq_indices = findall(x -> x < threshold, f2)
    
    return(any(x -> x > 0.02, pden2[low_freq_indices]))  # Set a significance threshold for low-frequency energy

end    # check_low_frequency_energy()


# Main analysis function
function analyse_spectra(df)
############################
    
    results = DataFrame(Date=[], peak_frequency=Float64[], peak_type=String[], second_peak_frequency=Float64[], 
        second_peak_value=Float64[], min_frequency=Float64[], min_value=Float64[], low_frequency_warning=Bool[])
    
    for i in 1:nrow(df)
        
        date = df.Date[i]
        pden2 = df.Pden2[i]
        f2 = df.f2[i]
        
        # Find local maxima and minima
        local_maxima, local_minima = find_local_extrema(pden2)
        
        # Identify the peak of the spectra
        peak_index = argmax(pden2)
        peak_frequency = f2[peak_index]
        peak_value = pden2[peak_index]
        
        # Calculate PM spectrum value at peak frequency
        pm_peak_value = pierson_moskowitz_spectrum(peak_frequency, peak_frequency)
        
        second_peak_frequency = NaN
        second_peak_value = NaN
        min_frequency = NaN
        min_value = NaN

##        println(date," ",peak_frequency," ",peak_value," ",pm_peak_value)
        
        if peak_value > pm_peak_value
            
            peak_type = "Wind Sea"
            # Analyse values to the left of the peak
            left_maxima = local_maxima[local_maxima .< peak_index]
            left_minima = local_minima[local_minima .< peak_index]
            
            if !isempty(left_maxima)
                second_peak_index = left_maxima[argmax(pden2[left_maxima])]
                second_peak_frequency = f2[second_peak_index]
                second_peak_value = pden2[second_peak_index]
                
                if !isempty(left_minima)
                    
                    min_between_peaks = left_minima[(left_minima .> second_peak_index) .& (left_minima .< peak_index)]
                    if !isempty(min_between_peaks)
                        min_index = min_between_peaks[argmin(pden2[min_between_peaks])]
                        min_frequency = f2[min_index]
                        min_value = pden2[min_index]
                    end
                    
                end
                
            end
            
        else
            
            peak_type = "Ocean Swell"
            # Analyise values to the right of the peak
            right_maxima = local_maxima[local_maxima .> peak_index]
            right_minima = local_minima[local_minima .> peak_index]
            
            if !isempty(right_maxima)
                
                second_peak_index = right_maxima[argmax(pden2[right_maxima])]
                second_peak_frequency = f2[second_peak_index]
                second_peak_value = pden2[second_peak_index]
                
                if !isempty(right_minima)
                    
                    min_between_peaks = right_minima[(right_minima .> peak_index) .& (right_minima .< second_peak_index)]
                    if !isempty(min_between_peaks)
                        min_index = min_between_peaks[argmin(pden2[min_between_peaks])]
                        min_frequency = f2[min_index]
                        min_value = pden2[min_index]
                    end
                    
                end
                
            end
            
        end
        
        # Check for low-frequency energy warning
        low_frequency_warning = check_low_frequency_energy(pden2, f2)
        
        push!(results, (date, peak_frequency, peak_type, second_peak_frequency, second_peak_value, min_frequency, min_value, low_frequency_warning))
    end
    
    return results

end    # analyse_spectra()


# Function to calculate Hm0
function calculate_Hm0(frequencies, spectral_values)
###############################################
    
    # Trapezoidal integration to calculate the area under the spectral curve
    area = sum(0.5 * (frequencies[2:end] .- frequencies[1:end-1]) .* (spectral_values[2:end] .+ spectral_values[1:end-1]))

    # Calculate Hm0
    Hm0 = 4 * sqrt(area)
    
    return(Hm0)

end    # calc_Hmo()


# Function to calculate Tp
function calculate_Tp(frequencies, spectral_values)
###################################################
    
    # Find the peak frequency
    peak_index = argmax(spectral_values)
    peak_frequency = frequencies[peak_index]
    
    # Calculate Tp
    Tp = 1 / peak_frequency
    
    return(Tp)
    
end    # calculate_Tp()


########################################################################################################################
########################################################################################################################
########################################################################################################################
display(HTML("<style>.jp-Cell { width: 120% !important; }</style>"))

# Analyse the spectra and get results
results = analyse_spectra(displacement_df);

# Join the two DataFrames on the Date column
results = innerjoin(results, displacement_df, on=:Date)

### Calculate the Hm0 for both Sea and Swell
Hm0 = Float64[]; Hm0_swell = Float64[]; Hm0_wind_sea = Float64[]
Tp = Float64[]; Tp_swell = Float64[]; Tp_wind_sea = Float64[]

for row in eachrow(results)
    
    frequencies = row.f2
    spectral_values = row.Pden2

    # calculate Hm0 and Tp over entire spectra
    push!(Hm0, calculate_Hm0(frequencies, spectral_values))
    push!(Tp, calculate_Tp(frequencies, spectral_values))
        
    sep_freq = row.min_frequency
    peak_type = row.peak_type
    
    if isnan(sep_freq)
        
        if peak_type == "Ocean Swell"
            # Entire spectrum is Ocean Swell
            push!(Hm0_swell, calculate_Hm0(frequencies, spectral_values))
            push!(Hm0_wind_sea,NaN)
        elseif peak_type == "Wind Sea"
            # Entire spectrum is Wind Sea
            push!(Hm0_swell,NaN)
            push!(Hm0_wind_sea,calculate_Hm0(frequencies, spectral_values))
        else
            push!(Hm0_swell,NaN)
            push!(Hm0_wind_sea,NaN)
        end
        
    else
        
        # Find the indices for Ocean Swell and Wind Sea
        swell_indices = findall(x -> x <= sep_freq, frequencies)
        wind_sea_indices = findall(x -> x > sep_freq, frequencies)
        
        # Separate the spectral values and frequencies
        swell_frequencies = frequencies[swell_indices]
        swell_spectral_values = spectral_values[swell_indices]
        wind_sea_frequencies = frequencies[wind_sea_indices]
        wind_sea_spectral_values = spectral_values[wind_sea_indices]
        
        # Calculate Hm0 and Tp for Ocean Swell and Wind Sea
        push!(Hm0_swell,calculate_Hm0(swell_frequencies,swell_spectral_values))
        push!(Hm0_wind_sea,calculate_Hm0(wind_sea_frequencies, wind_sea_spectral_values))
        
    end
    
end

# Extract peak_frequency values where peak_type is "Wind Sea"
peak_wind_sea_frequencies = results[results.peak_type .== "Wind Sea", :peak_frequency]
peak_ocean_swell_frequencies = results[results.peak_type .== "Ocean Swell", :peak_frequency]

# Extract second_peak_frequency values where peak_type is "Ocean Swell"
minor_ocean_swell_frequencies = results[results.peak_type .== "Ocean Swell", :second_peak_frequency]
minor_wind_sea_frequencies = results[results.peak_type .== "Wind Sea", :second_peak_frequency]

# Combine these frequencies into a single array
combined_wind_sea_frequencies = vcat(peak_wind_sea_frequencies, minor_ocean_swell_frequencies)
combined_ocean_swell_frequencies = vcat(peak_ocean_swell_frequencies, minor_wind_sea_frequencies)

Tp_wind_sea = 1 ./ combined_wind_sea_frequencies
Tp_swell = 1 ./ combined_ocean_swell_frequencies

# Do moving average filters over the Hm0 and Tp values
ma_filter = 3
ma_Hm0 = movavg(Hm0, ma_filter)
ma_wind_sea_Hm0 = movavg(Hm0_wind_sea, ma_filter)
ma_ocean_swell_Hm0 = movavg(Hm0_swell, ma_filter)

ma_Tp = movavg(Tp, ma_filter)
ma_wind_sea_Tp = movavg(Tp_wind_sea, ma_filter)
ma_ocean_swell_Tp = movavg(Tp_swell, ma_filter)

title = split(split(hxv_files[1],"\\")[end],"_")[1]*"_"*monthname(displacement_df.Date[1])*"_"*string(year(displacement_df.Date[1]))
plot_file = ".\\Plots\\"*title*"_sea_swell_split.png"

# Plot the Hm0 values
p1 = Plots.plot(results.Date, ma_Hm0.x, label="Combined Hm0", lc=:green, lw=:3, yaxis="Hm0 (m)", title=title)
p1 = Plots.plot!(results.Date, ma_wind_sea_Hm0.x, label="Wind Sea Hm0", lc=:blue, lw=:3)
p1 = Plots.plot!(results.Date, ma_ocean_swell_Hm0.x, label="Ocean Swell Hm0", lc=:red, lw=:3)

# Plot the Tp values
p2 = Plots.plot(results.Date, ma_Tp.x, label="Combined Tp", lc=:green, lw=:3, yaxis="Tp (s)")
p2 = Plots.plot!(results.Date, ma_wind_sea_Tp.x, label="Wind Sea Tp", lc=:blue, lw=:3)
p2 = Plots.plot!(results.Date, ma_ocean_swell_Tp.x, label="Ocean Swell Tp", lc=:red, lw=:3)

tm_tick = range(displacement_df.Date[1],displacement_df.Date[end],step=Day(1))
ticks = Dates.format.(tm_tick,"mm-dd")

plot_p1_p2 = Plots.plot(p1, p2, layout=(2,1), xlims=(displacement_df.Date[1],displacement_df.Date[end]), xticks=(tm_tick,ticks), xtickfontsize=8, xaxis="Date", 
    ylim=(0,Inf), ytickfontsize=8, titlefontsize=12, framestyle=:box, legendfontsize=12, legend=:topright,
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    leftmargin = 15Plots.mm, bottommargin = 5Plots.mm, grid=true, size=(1600,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(plot_p1_p2)

In [ ]:
using DataFrames

# Function to calculate Hm0
function calculate_Hm0(spectral_values, frequencies)
    # Trapezoidal integration to calculate the area under the spectral curve
    area = sum(0.5 * (frequencies[2:end] .- frequencies[1:end-1]) .* (spectral_values[2:end] .+ spectral_values[1:end-1]))
    # Calculate Hm0
    Hm0 = 4 * sqrt(area)
    return Hm0
end

# Function to calculate Tp
function calculate_Tp(spectral_values, frequencies)
    # Find the peak frequency
    peak_index = argmax(spectral_values)
    peak_frequency = frequencies[peak_index]
    # Calculate Tp
    Tp = 1 / peak_frequency
    return Tp
end

# Calculate Hm0 and Tp for Ocean Swell and Wind Sea parts
results1 = DataFrame(Date=[], Hm0_Ocean_Swell=Float64[], Hm0_Wind_Sea=Float64[], Tp_Ocean_Swell=Float64[], Tp_Wind_Sea=Float64[])

for row in eachrow(results)
    frequencies = row.f2
    spectral_values = row.Pden2
    sep_freq = row.min_frequency
    peak_type = row.peak_type
    
    if isnan(sep_freq)
        if peak_type == "Ocean Swell"
            # Entire spectrum is Ocean Swell
            Hm0_swell = calculate_Hm0(spectral_values, frequencies)
            Tp_swell = calculate_Tp(spectral_values, frequencies)
            Hm0_wind_sea = NaN
            Tp_wind_sea = NaN
        elseif peak_type == "Wind Sea"
            # Entire spectrum is Wind Sea
            Hm0_swell = NaN
            Tp_swell = NaN
            Hm0_wind_sea = calculate_Hm0(spectral_values, frequencies)
            Tp_wind_sea = calculate_Tp(spectral_values, frequencies)
        else
            Hm0_swell = NaN
            Tp_swell = NaN
            Hm0_wind_sea = NaN
            Tp_wind_sea = NaN
        end
    else
        # Find the indices for Ocean Swell and Wind Sea
        swell_indices = findall(x -> x <= sep_freq, frequencies)
        wind_sea_indices = findall(x -> x > sep_freq, frequencies)
        
        # Separate the spectral values and frequencies
        swell_frequencies = frequencies[swell_indices]
        swell_spectral_values = spectral_values[swell_indices]
        wind_sea_frequencies = frequencies[wind_sea_indices]
        wind_sea_spectral_values = spectral_values[wind_sea_indices]
        
        # Calculate Hm0 and Tp for Ocean Swell and Wind Sea
        Hm0_swell = calculate_Hm0(swell_spectral_values, swell_frequencies)
        Tp_swell = calculate_Tp(swell_spectral_values, swell_frequencies)
        Hm0_wind_sea = calculate_Hm0(wind_sea_spectral_values, wind_sea_frequencies)
        Tp_wind_sea = calculate_Tp(wind_sea_spectral_values, wind_sea_frequencies)
    end
    
    # Append the results
    push!(results1, (row.Date, Hm0_swell, Hm0_wind_sea, Tp_swell, Tp_wind_sea))
end

# Display the results
println(results)


In [ ]:
using Plots

gr()

display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

dates = displacement_df.Date
freqs = displacement_df.fhh[1]
spectra = hcat(displacement_df.Pden2...)
max_spec = maximum(spectra)

start_date =  first(dates)
last_date = last(dates)

# display plots to screen
tm_tick = range(start_date,last_date,step=Day(1))
ticks = Dates.format.(tm_tick,"dd")

#p1 = heatmap(first(displacement_df.Date) + Microsecond.(ceil.((spec.time) * 1000000)), spec.freq, power_spec, lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0.0,max_spec), levels=10, fill=true)
p1 = contourf(dates, freqs, spectra, lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)
#p1 = Plots.plot!(displacement_df.Date,noise_floors)
# draw grid lines on plot
for i in 0:0.1:0.6
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Day(1):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

ma_filter = 5
ma_Tp = movavg(Tp, ma_filter)
ma_wind_sea_Tp = movavg(results1.Tp_Wind_Sea, ma_filter)
ma_ocean_swell_Tp = movavg(results1.Tp_Ocean_Swell, ma_filter)

p1 = Plots.plot!(results1.Date, 1 ./ ma_ocean_swell_Tp.x, label="", lc=:yellow, lw=:0.5)
p1 = Plots.plot!(results.Date, 1 ./ ma_wind_sea_Tp.x, label="", lc=:lightgray, lw=:0.5)

#title = Dates.format(start_date, "yyyy-mm-dd") * " to " * Dates.format(last_date, "yyyy-mm-dd")
title = split(split(hxv_files[1],"\\")[end],"_")[1]*"_"*monthname(displacement_df.Date[1])*"_"*string(year(displacement_df.Date[1]))
plot_file = ".\\Plots\\"*title*"_monthly_spectrogram.png"

p1_plot = plot(p1, xlabel="Date", xlim=(start_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.4), ytickfontsize=8, 
        title=title, framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=true)

try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(p1_plot)

In [ ]:
# Function to replace NaN with 0 in the specified column of the DataFrame
function replace_nan_with_zero!(df::DataFrame, col::Symbol)
###########################################################
    
    df[!, col] = [replace(x -> isnan(x) ? 0 : x, df[!, col][i]) for i in 1:nrow(df)]

end


function find_peak_spectral_value_and_frequency(df)
###################################################       

    peak_values = Float64[]
    peak_frequencies = Float64[]
    
    for row ∈ eachrow(df)
        
        spectra = row.Pden2
        frequencies = row.f2
        peak_value = maximum(spectra)
        peak_index = argmax(spectra)
        peak_frequency = frequencies[peak_index]
        push!(peak_values, peak_value)
        push!(peak_frequencies, peak_frequency)
        
    end
    
    df.peak_value = peak_values
    df.peak_frequency = peak_frequencies
    
end    # find_peak_spectral_value_and_frequency()


# Calculate a PM spectrum for a given peak frequency
function pierson_moskowitz_spectrum(f, fp)
##########################################
    
    α = 0.0081
    g = 9.81
    
    return(α * g^2 * (2 * π)^-4 * f .^ -4 .* exp.(-5/4 * (fp ./ f) .^ 4))
    
end    # pierson_moskowitz_spectrum()


# Calculate a JONSWAP spectrum for a given peak frequency
function jonswap_spectrum(frequencies, fp)
##########################################

    α = 0.076  # Scaling factor
    γ = 3.3  # Peak enhancement factor
    g = 9.81  # Acceleration due to gravity (m/s^2)

    σ = [f > fp ? 0.09 : 0.07 for f ∈ frequencies]
    r = exp.(-((frequencies .- fp).^2) ./ (2 .* σ.^2 .* fp^2))
    s = α * g^2 * (2 * π)^(-4) .* frequencies.^(-5) .* exp.(-5/4 * (fp ./ frequencies).^4) .* γ.^r
    
    return(s)

end    # jonswap_spectrum()


# Determine whether record is Ocean Swell or Wind Sea by its relationnship to the PM spectrum
function classify_wave_events(df)
#################################    

    global classifications = String[]
    
    for row ∈ eachrow(df)
        
        spectra = row.Pden2
        frequencies = row.f2
        peak_value = row.peak_value
        peak_frequency = row.peak_frequency
        pm_spectrum = replace(pierson_moskowitz_spectrum(frequencies, peak_frequency), NaN => 0.0)
        sJONSWAP = replace(jonswap_spectrum(frequencies, peak_frequency), NaN => 0.0)

        pm_peak_value = maximum(pm_spectrum)

        ##########################################################################################################
        ## ALERT: I've noticed that the PM spectrum ratio method is not as effective as I had first thought.
        ## Consequently, I have added an additional test - it is unlikely that a Swell wave would be 6.5s or shorter.
        ## So, if the PM test says it is Ocean Swell but it's period is 6.5s or less, we will call it Wind Sea!
        ##########################################################################################################
        
        if (pm_peak_value >= peak_value) & (peak_frequency < 0.14)
            
            push!(classifications, "Ocean Swell")

        else
            
            push!(classifications, "Wind Sea")

        end
        
    end
    
    df.classification = classifications
    
end    # classify_wave_events()


function find_separation_point(df)
##################################

    separation_points = Float64[]
    
    for row ∈ eachrow(df)
        
        spectra = row.Pden2
        peak = argmax(spectra)
        frequencies = row.f2
        peak_value = row.peak_value
        peak_frequency = row.peak_frequency

        if row.classification == "Wind Sea"

            flush(stdout)
            
            threshold = 0.1 * peak_value
            
            # Identify the first peak above the threshold
            peak_indices = findall(x -> x > threshold, spectra)
            
            if isempty(peak_indices)
                push!(separation_points, nothing)
                continue
            end
            
            first_peak_index = peak_indices[1]
            
            # Find the minimum value between this peak and the original peak
            original_peak_index = argmax(spectra)
            
            if first_peak_index >= original_peak_index
                push!(separation_points, 0)
                continue
            end
            
            interval_spectra = spectra[first_peak_index:original_peak_index]
            interval_frequencies = frequencies[first_peak_index:original_peak_index]
            min_value, min_index = findmin(interval_spectra)
            separation_frequency = interval_frequencies[min_index]
            
            # Find the peak value between the start of the spectra and this minimum value
            start_interval_spectra = spectra[1:min_index]
            start_interval_frequencies = frequencies[1:min_index]
            swell_peak_value = maximum(start_interval_spectra)
            swell_peak_index = argmax(start_interval_spectra)
            swell_peak_frequency = start_interval_frequencies[swell_peak_index]
            
            push!(separation_points, separation_frequency)
##            println("Swell peak value: ", swell_peak_value, ", Swell peak frequency: ", swell_peak_frequency)
            
        else    # For Ocean Swell classification

            flush(stdout)
            
            # If the event is Ocean Swell, find the separation point
            end_index = findfirst(x -> x >= 0.4, frequencies)
            
            if isnothing(end_index)
                push!(separation_points, 0)
                continue
            end

            interval_spectra = spectra[peak:end_index]
            interval_frequencies = frequencies[peak:end_index]
            min_value, min_index = findmin(interval_spectra)
            separation_frequency = interval_frequencies[min_index]
            
            # Find the peak value between the minimum value and the end of the spectra
            end_interval_spectra = spectra[min_index:end_index]
            end_interval_frequencies = frequencies[min_index:end_index]
            swell_peak_value = maximum(end_interval_spectra)
            swell_peak_index = argmax(end_interval_spectra)
            swell_peak_frequency = end_interval_frequencies[swell_peak_index]
            
            push!(separation_points, separation_frequency)
##            println("Swell peak value: ", swell_peak_value, ", Swell peak frequency: ", swell_peak_frequency)
            
        end
        
    end
    
    df.separation_point = separation_points
    
end    # find_separation_point()

split_df = deepcopy(displacement_df)

# Replace NaN with 0
replace_nan_with_zero!(split_df, :Pden2)

# Apply the functions
find_peak_spectral_value_and_frequency(split_df);
classify_wave_events(split_df);
find_separation_point(split_df);

println("Done")


In [ ]:
using Plots

##split_df = deepcopy(displacement_df)
iii = 1152

peak = argmax(replace(results.Pden2[iii], NaN => 0.0))
f = results.f2[iii]
f0 = f[peak]
sPM = pierson_moskowitz_spectrum(f, f0)
#sJONSWAP = jonswap_spectrum(f, f0)

title = Dates.format(results.Date[iii], "yyyy-mm-dd HH:MM")
p1 = Plots.plot(results.f2[iii], results.Pden2[iii], lw=:4, label="")
#p1 = Plots.plot!(results.fhh[iii],results.Chh[iii],label="")
p1 = Plots.plot!(f, sPM, lw=:3, label="PM spectrum")
#p1 = Plots.plot!(f, sJONSWAP, lw=:3, label="JONSWAP spectrum")
p1 = Plots.vline!([f0], label="Peak frequency")
#p1 = Plots.vline!([split_df.separation_point[iii]], ls=:dash, label="Separation Point")
#p1 = Plots.hline!([0.1 * split_df.peak_value[iii]], label="Threshold")

##@printf("Peak value = %0.2f Peak Frequency = %0.2f Separation Point = %0.2f (%0.2fs) Classification = ##%s",split_df.peak_value[iii],split_df.peak_frequency[iii],split_df.separation_point[iii],1/split_df.separation_point[iii],split_df.classification[iii])

p1_plot = Plots.plot(p1, layout=(1,1), xlims=(0,0.64), ylims=(0,results.Pden2[iii][peak]*1.1), xaxis="Frequency (Hertz)", xminorticks=10,
    ytickfontsize=8, title=title, titlefontsize=10, framestyle = :box, guidefontsize=10,
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    leftmargin = 15Plots.mm, bottommargin = 5Plots.mm, grid=true, size=(1600,600), gridlinewidth=0.5, gridstyle=:dot, gridα=1)

display(p1_plot)

In [ ]:
Plots.plot(displacement_df.Heave[911], xlims=(0,500))

### Plot Sea and Swell time-series

In [ ]:
using Plots

display(HTML("<style>.jp-Cell { width: 150% !important; }</style>"))

#p1 = Plots.plot(displacement_df.Date, Hm0_combined, lw=:4, label="Combined")
p1 = Plots.plot(split_df.Date, Hm0_sea, lw=:4, label="Wind Sea")
p1 = Plots.plot!(split_df.Date, Hm0_swell, lw=:4, label="Ocean Swell")

tm_tick = range(split_df.Date[1],split_df.Date[end],step=Day(1))
ticks = Dates.format.(tm_tick,"mm-dd")

title = split(split(hxv_files[1],"\\")[end],"_")[1]*"_"*monthname(displacement_df.Date[1])*"_"*string(year(displacement_df.Date[1]))
plot_file = ".\\Plots\\"*title*"_sea_swell_split.png"

p1_plot = Plots.plot(p1, layout=(1,1), xlims=(split_df.Date[1],split_df.Date[end]), xticks=(tm_tick,ticks), xtickfontsize=8, xaxis="Date", 
    yaxis="Wave height (m)", ytickfontsize=8, title=title, titlefontsize=12, framestyle=:box, legendfontsize=15,
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    leftmargin = 15Plots.mm, bottommargin = 5Plots.mm, grid=true, size=(1600,600), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)


try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(p1_plot)



In [ ]:
using Plots
using Tk

w = Toplevel("Select Date", 235, 800)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)

lb = Treeview(f1, Dates.format.(split_df.Date, "yyyy-mm-dd HH:MM"))
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global target_date = get_value(lb);
    iii = findall(Dates.format.(split_df.Date, "yyyy-mm-dd HH:MM").== target_date)[1]
##    println(target_date, iii)
    
    peak = argmax(replace(split_df.Pden2[iii], NaN => 0.0))
    f = split_df.f2[iii]
    f0 = f[peak]
    sPM = pierson_moskowitz_spectrum(f, f0)
    sJONSWAP = jonswap_spectrum(f, f0)
    
    title = Dates.format(split_df.Date[iii], "yyyy-mm-dd HH:MM")
    p1 = Plots.plot(split_df.f2[iii], split_df.Pden2[iii], lw=:4, label="", xlims=(0,0.64), ylims=(0,split_df.Pden2[iii][peak]*1.1), xaxis="Frequency (Hertz)", xminorticks=10,title=title,)
    #p1 = Plots.plot!(split_df.fhh[iii],split_df.Chh[iii],label="")
    p1 = Plots.plot!(f, sPM, lw=:3, label="PM spectrum")
    p1 = Plots.plot!(f, sJONSWAP, lw=:3, label="JONSWAP spectrum")
    p1 = Plots.vline!([f0], label="Peak frequency")
    p1 = Plots.vline!([split_df.separation_point[iii]], ls=:dash, label="Separation Point")
    p1 = Plots.hline!([0.1 * split_df.peak_value[iii]], label="Threshold")

    heave_time = split_df.Date[iii] .+ (0:length(split_df.Heave[iii])-1) .* Microsecond.(1/1.28 * 1000000)
    tm_tick = range(heave_time[1],heave_time[end],step=Minute(1))
    ticks = Dates.format.(tm_tick,"MM")

    p2 = Plots.plot(heave_time, split_df.Heave[iii], c="blue", lw=0.5, label="", xticks=(tm_tick,ticks), xtickfontsize=8, xlims=(heave_time[1],heave_time[end]), xaxis="Heave (m)")

    @printf("Peak value = %0.2fm²/Hz. Peak Frequency = %0.2fHz. Separation Point = %0.2fHz. (%0.2fs) Hₘ₀ Sea = %0.2fm Hₘ₀ Swell = %0.2fm Classification = %s",
        split_df.peak_value[iii],split_df.peak_frequency[iii],split_df.separation_point[iii],1/split_df.separation_point[iii], split_df.Hm0_Sea[iii], split_df.Hm0_Swell[iii], split_df.classification[iii])
    
    p1_plot = Plots.plot(p1, p2, layout=(2,1),
        ytickfontsize=8, titlefontsize=10, framestyle = :box, guidefontsize=10,
        fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
        leftmargin = 15Plots.mm, bottommargin = 5Plots.mm, grid=true, size=(1600,800), gridlinewidth=0.5, gridstyle=:dot, gridα=1)
    
    display(p1_plot)

end

In [ ]:
using Statistics
using LinearRegression

function spectral_slope(frequencies, spectrum)
    log_freqs = log.(frequencies)
    log_spectrum = log.(spectrum)
    model = linregress(log_freqs, log_spectrum)
    slope = model.coeffs[2]
    intercept = model.coeffs[1]
    return slope, intercept
end


function spectral_width(frequencies, spectrum)
    mean_freq = sum(frequencies .* spectrum) / sum(spectrum)
    width = sqrt(sum(((frequencies .- mean_freq) .^ 2) .* spectrum) / sum(spectrum))
    return width
end

function wave_age(frequencies, spectrum, wind_speed)
    peak_index = argmax(spectrum)
    peak_frequency = frequencies[peak_index]
    wave_phase_speed = 1.56 / peak_frequency  # Approximation for deep water waves
    age = wave_phase_speed / wind_speed
    return age
end

iii = 15

frequencies = split_df.f2[iii]
spectrum = split_df.Pden2[iii]

slope = spectral_slope(frequencies, spectrum)
println("Spectral Slope: ", slope)


width = spectral_width(frequencies, spectrum)
println("Spectral Width: ", width)

wind_speed = 10  # Example wind speed in m/s
age = wave_age(frequencies, spectrum, wind_speed)
println("Wave Age: ", age)
